In [1]:
import numpy as np
from scipy.io import loadmat
from pybalu.feature_selection import clean, sfs
from pybalu.feature_transformation import normalize
from sklearn.neighbors import KNeighborsClassifier

#### Común

Las siguientes funciones se definen para poder dividir el set de datos en los sets de entrenamiento y testing de acuerdo a lo pedido, esto es, que se utilice el primer 80% de las muestras de cada clase para entrenar y el 20% restante para las pruebas.

Con ese objetivo se crea el método `train_test_split` para ser consistente con el método ampliamente conocido de sklearn.

In [2]:
def split(array, proportion):
    splitting_point = round(array.shape[0] * (1 - proportion))
    return array[:splitting_point], array[splitting_point:]

def train_test_split(features, labels, proportion):
    classes = np.unique(labels)
    features_train = []
    features_test = []
    labels_train = []
    labels_test = []
    for cl in classes:
        class_train, class_test = split(features[(labels == cl)[:, 0]], proportion)
        class_train_labels, class_test_labels = split(labels[labels == cl], proportion)
        features_train.append(class_train)
        features_test.append(class_test)
        labels_train.append(class_train_labels)
        labels_test.append(class_test_labels)
    f_train = features_train[0]
    f_test = features_test[0]
    l_train = labels_train[0]
    l_test = labels_test[0]
    for x in range(1, classes.size):
        f_train = np.vstack((f_train, features_train[x]))
        f_test = np.vstack((f_test, features_test[x]))
        l_train = np.hstack((l_train, labels_train[x]))
        l_test = np.hstack((l_test, labels_test[x]))
    return f_train, f_test, l_train.reshape(l_train.size, 1), l_test.reshape(l_test.size, 1)

## Tortillas

Para esta sección cabe recordar que se pide explícitamente eliminar las características de posición por lo sesgado del _dataset_, de modo que se eliminan de antemano.

In [3]:
data = loadmat('set04-tortillas.mat')

In [4]:
excluded = ['center of grav i      ',
            'center of grav j      ',
            'Ellipse-centre i      ',
            'Ellipse-centre j      ']

In [5]:
indexes = []
for feature in excluded:
    indexes.append(np.where(data['fn'] == feature)[0][0])
    
position_indexes = np.array(indexes)
non_position_indexes = np.setdiff1d(np.array(range(data['fn'].size)), position_indexes)

In [6]:
X = data['f'][:, non_position_indexes]
y = data['d']

Se dividen los datos en la proporción pedida:

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2)

Se hace `clean` de los datos, cuidando de aplicar los índices obtenidos también al set de testing:

In [8]:
clean_indexes = clean(X_train)

In [9]:
X_train_cleaned = X_train[:, clean_indexes]
X_test_cleaned = X_test[:, clean_indexes]

Se realiza normalización, también aplicandola al set de testing:

In [10]:
X_train_normalized, a, b = normalize(X_train_cleaned)
X_test_normalized = X_test_cleaned * a + b

Se realiza una selección de características con `SFS`, se probó con diferentes cantidades de características, pero ya con 10 se obtenían resultados perfectos. También se aplican los índices obtenidos al set de pruebas:

In [11]:
N_FEATURES = 10
selected_indexes = sfs(X_train_normalized, y_train, n_features=N_FEATURES, method="fisher", show=True)

Selecting Features: 100%|██████████| 10.0/10.0 [00:01<00:00, 6.81 features/s]


In [12]:
X_train_selected = X_train_normalized[:, selected_indexes]
X_test_selected = X_test_normalized[:, selected_indexes]

Para finalmente determinar la precisión del método aplicado se prueba directamente con un `KNN` de un vecino, y el resultado es de un 100%. Sin embargo, también se probó añadiendo `PCA` y luego clasificando con `KNN` o directamente con `LDA`, sin embargo como ya se obtenía un 100% sin realizar lo anterior se dejó este método.

In [13]:
knn_sfs = KNeighborsClassifier(n_neighbors=1)
knn_sfs.fit(X_train_selected, y_train)
print(f'{knn_sfs.score(X_test_selected, y_test) * 100}%')

100.0%


## Caras

Se utiliza en primer lugar el comando `%reset -f array` para que todos los arreglos de numpy se _reseteen_ de modo de evitar por cualquier razón utilizar datos de la sección anterior.

In [14]:
%reset -f array

Como se verá en los experimentos, esta vez si mejora el rendimiento al utilizar `PCA`, `LDA` o `QDA`, por lo que se incluyen

In [15]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

El procedimiento es muy similar a lo anterior por lo que no se comentará nuevamente hasta las partes donde difiera.

In [16]:
data = loadmat('set05-face-detection.mat')

In [17]:
X = data['f']
y = data['d']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2)

In [19]:
clean_indexes = clean(X_train)

In [20]:
X_train_cleaned = X_train[:, clean_indexes]
X_test_cleaned = X_test[:, clean_indexes]

In [21]:
X_train_normalized, a, b = normalize(X_train_cleaned)
X_test_normalized = X_test_cleaned * a + b

In [22]:
N_FEATURES = 9
selected_indexes = sfs(X_train_normalized, y_train, n_features=N_FEATURES, method="fisher", show=True)

Selecting Features: 100%|██████████| 9.00/9.00 [00:00<00:00, 9.15 features/s]


In [23]:
X_train_selected = X_train_normalized[:, selected_indexes]
X_test_selected = X_test_normalized[:, selected_indexes]

Tras realizar la selección con `SFS` con 9 características (luego de haber intentado con diversos números), se obtienen resultados muy satisfactorios, sin embargo, no perfectos:

In [24]:
knn_sfs = KNeighborsClassifier(n_neighbors=2)
knn_sfs.fit(X_train_selected, y_train)
print(f'{knn_sfs.score(X_train_selected, y_train) * 100}%')
print(f'{knn_sfs.score(X_test_selected, y_test) * 100}%')

98.5781990521327%
90.56603773584906%


Es importante notar que no tiene sentido hacer PCA de 9 componentes a algo que de por si tiene 9 dimensiones ya que no se juntaría la información de ningún par de ellas en otro eje nuevo.

Por otro lado, se prueba que con menos no hay mejoras significativas del rendimiento, lo que hace sentido, porque de lo contrario, significaría que el método sigue estando sobreajustado a los datos, pero con 9 características desde las más de mil iniciales, es poco probable.

In [25]:
pca = PCA(n_components=6)
X_train_pcaed = pca.fit_transform(X_train_selected)
X_test_pcaed = pca.transform(X_test_selected)

In [26]:
knn_pca = KNeighborsClassifier(n_neighbors=2)
knn_pca.fit(X_train_pcaed, y_train)
knn_pca.score(X_test_pcaed, y_test)
print(f'{knn_pca.score(X_train_pcaed, y_train) * 100}%')
print(f'{knn_pca.score(X_test_pcaed, y_test) * 100}%')

98.10426540284361%
88.67924528301887%


Como se aprecia, con `PCA` de 2 vecinos no se mejora en training, por lo que no es natural elegir este método por sobre el otro. Ahora, se muestran las pruebas con `LDA` o `QDA` en vez de `KNN` directo tras la selección o después de la aplicación de `PCA`:

In [27]:
lda = LDA(solver='svd', n_components=10)
lda.fit(X_train_selected, y_train)
print(f'{lda.score(X_train_selected, y_train) * 100}%')
print(f'{lda.score(X_test_selected, y_test) * 100}%')

97.6303317535545%
92.45283018867924%


In [28]:
lda = LDA(solver='lsqr', shrinkage='auto', n_components=10)
lda.fit(X_train_pcaed, y_train)
print(f'{lda.score(X_train_pcaed, y_train) * 100}%')
print(f'{lda.score(X_test_pcaed, y_test) * 100}%')

92.89099526066352%
90.56603773584906%


Como se observa el rendimiento en entrenamiento no mejora con `LDA`, por lo que no es natural elegir esto, y si se eligiera por el desempeño en testing se estaría haciendo algo que no tiene sentido (porque no siempre se tiene acceso a esos datos).

Ahora solo resta probar con el otro analisis de discriminante, `QDA`, en vez del lineal. Nuevamente sobre los datos tras la selección de características:

In [29]:
qda = QDA()
qda.fit(X_train_selected, y_train)
print(f'{qda.score(X_train_selected, y_train) * 100}%')
print(f'{qda.score(X_test_selected, y_test) * 100}%')

98.10426540284361%
98.11320754716981%


In [30]:
qda = QDA()
qda.fit(X_train_pcaed, y_train)
print(f'{qda.score(X_train_pcaed, y_train) * 100}%')
print(f'{qda.score(X_test_pcaed, y_test) * 100}%')

93.8388625592417%
94.33962264150944%


Como en los 3 casos con clasificadores distintos a `KNN` no se obtuvo mejores resultados en training lo natural es decantarse por la primera opción, previa a PCA. Sin embargo, se puede argumentar que las últimas son más robustas en base a que pueden utilizar menos componentes, pero nunca argumentarlo en base a los resultados de testing, puesto que es ilegal.

Sin embargo, para clasificar ahora si se puede usar KNN con 1 vecino, ya que antes se usaba con 2 para evitar tener 100% siempre con datos de training ya que eso no permitía comparar.

In [31]:
knn_sfs = KNeighborsClassifier(n_neighbors=1)
knn_sfs.fit(X_train_selected, y_train)
print(f'{knn_sfs.score(X_test_selected, y_test) * 100}%')

94.33962264150944%


Habiendo hecho la elección con los _scores_ de training, se ve que se llega a un resultado bastante bueno en testing, inferior al de QDA pero eso sería con información que no se tiene a priori e ilegal, por lo que el resultado final es de **94.3%**, tras SFS con KNN de 1 vecino.

## Géneros

Para esta sección nuevamente se tiene cuidado de no utilizar los mismos arreglos, y se procede de manera similar.

In [32]:
%reset -f array

In [33]:
data = loadmat('set06-gender.mat')

In [34]:
X = data['f']
y = data['d']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2)

In [36]:
clean_indexes = clean(X_train)

In [37]:
X_train_cleaned = X_train[:, clean_indexes]
X_test_cleaned = X_test[:, clean_indexes]

In [38]:
X_train_normalized, a, b = normalize(X_train_cleaned)
X_test_normalized = X_test_cleaned * a + b

In [39]:
N_FEATURES = 12
selected_indexes = sfs(X_train_normalized, y_train, n_features=N_FEATURES, method="fisher", show=True)

Selecting Features: 100%|██████████| 12.0/12.0 [00:01<00:00, 9.56 features/s]


In [40]:
X_train_selected = X_train_normalized[:, selected_indexes]
X_test_selected = X_test_normalized[:, selected_indexes]
selected_indexes

array([233,  59, 265, 263, 301,  51, 106, 131,   6, 164, 112,  28])

In [41]:
knn_sfs = KNeighborsClassifier(n_neighbors=2)
knn_sfs.fit(X_train_selected, y_train)
print(f'{knn_sfs.score(X_train_selected, y_train) * 100}%')
print(f'{knn_sfs.score(X_test_selected, y_test) * 100}%')

98.97540983606558%
62.295081967213115%


El resultado es bastante bueno en training, pero con PCA en training se puede mejorar

In [42]:
pca = PCA(n_components=3)
X_train_pcaed = pca.fit_transform(X_train_selected)
X_test_pcaed = pca.transform(X_test_selected)

In [43]:
knn_pca = KNeighborsClassifier(n_neighbors=2) # uso 2 porque sino sería 100% en training
knn_pca.fit(X_train_pcaed, y_train)
print(f'{knn_pca.score(X_train_pcaed, y_train) * 100}%')
print(f'{knn_pca.score(X_test_pcaed, y_test) * 100}%')

99.38524590163934%
62.295081967213115%


Sin embargo, similar a lo realizado en la tarea anterior, por como se comporta `SFS` una opción es probar con combinaciones de largo inferior, es decir subconjuntos de índices.

In [44]:
from itertools import combinations

In [45]:
r = []
for i in range(6, N_FEATURES + 1):
    indexes = combinations(selected_indexes[:i], i - 5)
    for x in indexes:
        xtrs = X_train_normalized[:, x]
        xtts = X_test_normalized[:, x]
        k = KNeighborsClassifier(n_neighbors=2) # con 2 porque sino siempre es 100% con training
        k.fit(xtrs, y_train)
        r.append([k.score(xtrs, y_train), i, x]) # xtrs es training.
best_selected_indexes = max(r)[2]
max(r)

[0.9979508196721312, 12, (265, 301, 51, 106, 131, 112, 28)]

Para hacer la elección es necesario utilizar el set de training porque elegir los parámetros usando el set de testing es _ilegal_. Las comparciones a posterior se realizan con los scores de testing pero la elección de parámetros no. Y es así para todo el código.

Si no se desea ejecutar el código para no computar las combinaciones, comentarlo y descomentar la línea del siguiente

In [46]:
best_selected_indexes
# best_selected_indexes = np.array([265, 301, 51, 106, 131, 112, 28])

(265, 301, 51, 106, 131, 112, 28)

Ahora realizamos el procedimiento de `KNN` con lo obtenido, y se verá si vale la pena aplicar `PCA` o utilizar otro método.

In [47]:
X_train_selected = X_train_normalized[:, best_selected_indexes]
X_test_selected = X_test_normalized[:, best_selected_indexes]

In [48]:
knn_sfs = KNeighborsClassifier(n_neighbors=2)
knn_sfs.fit(X_train_selected, y_train)
print(f'{knn_sfs.score(X_train_selected, y_train) * 100}%')
print(f'{knn_sfs.score(X_test_selected, y_test) * 100}%')

99.79508196721312%
79.50819672131148%


Como se observa, en training se mejora un poco respecto al resultado anterior, esto es de 99.4 a 99.8. 

Ahora con `PCA`:

In [51]:
pca = PCA(n_components=6)
X_train_pcaed = pca.fit_transform(X_train_selected)
X_test_pcaed = pca.transform(X_test_selected)
knn_pca = KNeighborsClassifier(n_neighbors=2)
knn_pca.fit(X_train_pcaed, y_train)
print(f'{knn_pca.score(X_train_pcaed, y_train) * 100}%')
print(f'{knn_pca.score(X_test_pcaed, y_test) * 100}%')

99.79508196721312%
81.14754098360656%


Sin embargo, el rendimiento no mejora para cualquier valor de `N` (componentes), en training, pero si se mantiene practicamente igual reduciendo la cantidad de componentes, lo que es positivo considerando la robustez.

Sin embargo, se observa que en testing es mejor, pero no habría manera de discriminar entre esto y lo anterior (sin PCA) porque los rendimientos son iguales.

Ahora, por otro lado, cabe señalar que si bien para elegir un mejor clasificador es buena idea utilizar k>1 en los vecinos cercanos porque sino se iguala a si mismo, es buena idea para testing utilizar el vecino más cercano, por lo que se ilustrará el rendimiento final de los casificadores con k=1, ya que con un set de datos diferente al que se usó para entrenar **sí tiene sentido** hacerlo.

In [52]:
# SIN PCA
knn_sfs = KNeighborsClassifier(n_neighbors=1)
knn_sfs.fit(X_train_selected, y_train)
print(f'{knn_sfs.score(X_test_selected, y_test) * 100}%')
# CON PCA
knn_pca = KNeighborsClassifier(n_neighbors=1)
knn_pca.fit(X_train_pcaed, y_train)
print(f'{knn_pca.score(X_test_pcaed, y_test) * 100}%')

86.0655737704918%
88.52459016393442%


Como en ambos se obtuvo el mismo resultado en training, es imposible saber sin acceso a los datos de testing cual habría desempeñado mejor, por lo que se opta por considerar el **86.06%** como el mejor resultado.